【Task 5】 模型调优（2天）  
任务5：使用网格搜索法对5个模型进行调优（调参时采用五折交叉验证的方式），并进行模型评估，记得展示代码的运行结果。   
时间：2天  

In [39]:
import pandas as pd 
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
import xgboost as xgb
import lightgbm as lgb
from pandas import DataFrame,Series

df = pd.read_csv('data.csv',encoding = 'gbk')

"""
数据处理
"""

###删除无关特征###
df.drop(['Unnamed: 0','custid','trade_no','bank_card_no' ,'source','id_name'],inplace =True,axis = 1)

###数据类型转换###（主要针对 obeject（文字类） 
df['reg_preference_for_trad'].fillna('其他城市',inplace = True)
df['reg_preference_for_trad'].replace({'一线城市':1,'二线城市':2,'三线城市':3,'境外':4,'其他城市':5},inplace = True)

# 处理日期格式 'latest_query_time', 'loans_latest_time'(暂时去掉？？？)

df.drop(['latest_query_time', 'loans_latest_time'],inplace =True,axis = 1)


###缺失值处理###
df['student_feature'].fillna(0,inplace=True) 
for i in df.columns:
    df[i].fillna(df[i].mode()[0],inplace = True)  #加[0]是因为众数可能有多个，返回不是一个数字

### 评估方法一：切分数据集

In [40]:
###切分数据集###

y=df['status']
x=df.drop('status',axis=1)
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.3,random_state = 2018)   

features = x_train.columns #目前是83个特征
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [41]:
#随机森林
clf_rf = RandomForestClassifier()
clf_rf.fit(x_train, y_train)
rf_y_pred = clf_rf.predict(x_test)

#评估 
ans = classification_report(y_test,rf_y_pred,digits=5)
print(ans)

             precision    recall  f1-score   support

          0    0.78588   0.92790   0.85101      1068
          1    0.53614   0.24791   0.33905       359

avg / total    0.72306   0.75683   0.72221      1427



### 评估方法二：交叉验证

In [42]:
#  使用交叉验证:直接就可以验证模型，这里用全部数据集而不是训练集
from sklearn.model_selection import cross_val_predict,cross_val_score

# #先做标准化  这里做不做都一样
scaler = StandardScaler()
x = scaler.fit_transform(x)
x= scaler.fit_transform(x)

model_rf = RandomForestClassifier()
score_rf = cross_val_score(model_rf, x,y,cv = 5)
pred_rf = cross_val_predict(model_rf, x,y,cv = 5)
print(score_rf,score_rf.mean())
ans1 = classification_report(y,pred_rf,digits=5)
print(ans1)

[ 0.77521008  0.78338591  0.77707676  0.76526316  0.77473684] 0.775134550981
             precision    recall  f1-score   support

          0    0.79206   0.94664   0.86248      3561
          1    0.61847   0.25817   0.36428      1193

avg / total    0.74850   0.77387   0.73746      4754



## 模型微调

### 方法一：网格搜索

In [46]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
rf_model = RandomForestClassifier()
grid_search = GridSearchCV(rf_model, param_grid, cv=5)
grid_search.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_features': [2, 4, 6, 8], 'n_estimators': [3, 10, 30]}, {'bootstrap': [False], 'max_features': [2, 3, 4], 'n_estimators': [3, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [52]:
# 搜索结果
print('最高得分：%.5f'% grid_search.best_score_)
print('最优参数：n_estimators:{} max_features:{} bootstrap:{}'.format(grid_search.best_estimator_.n_estimators,\
                                                        grid_search.best_estimator_.max_features,grid_search.best_estimator_.bootstrap))

最高得分：0.78776
最优参数：n_estimators:30 max_features:8 bootstrap:True
